In [2]:
import torch
import torch.nn as nn
import numpy as np
\If you are using ccx loss, first get the feature map using vgg19 for both images and then put it in the function

In [9]:
class CCX_loss(nn.Module):
    def __init__(self,h=0.5,eps=1e-6):
        super(CCX_loss, self).__init__()
        self.h = h
        self.eps = eps
    def forward(self, x, y):
        N, C, H, W = x.size()

        y_mu = y.mean(3).mean(2).mean(0).reshape(1, -1, 1, 1)

        x_centered = x - y_mu
        y_centered = y - y_mu
        x_normalized = x_centered / torch.norm(x_centered, p=2, dim=1, keepdim=True)
        y_normalized = y_centered / torch.norm(y_centered, p=2, dim=1, keepdim=True)

        x_normalized = x_normalized.reshape(N, C, -1)                               
        y_normalized = y_normalized.reshape(N, C, -1)                               
        cosine_sim = torch.bmm(x_normalized.transpose(1, 2), y_normalized)          

        d = 1 - cosine_sim                                  
        d_min, _ = torch.min(d, dim=2, keepdim=True)        

        d_tilde = d / (d_min + 1e-5)

        w = torch.exp((1 - d_tilde) / self.h)

        ccx_ij = w / torch.sum(w, dim=2, keepdim=True)       
        ccx = torch.mean(torch.max(ccx_ij, dim=1)[0], dim=1) 
        ccx_loss = torch.mean(-torch.log(ccx + self.eps))
        return ccx_loss

In [5]:
img0 = torch.ones(1,3,64,64)
img1 = torch.zeros(1,3,64,64)

In [6]:
loss(img0,img1)

tensor(1.)